In [65]:
import pandas as pd

df = pd.read_pickle('data/saeb_treated.pkl')
df.head()

,ano,sigla_uf,escola_publica,id_aluno,situacao_censo,disciplina,proficiencia,erro_padrao,proficiencia_saeb,erro_padrao_saeb,...,frequenta_festas_comunidade,gosta_estudar_disciplina,faz_licao_casa,correcao_licao_casa,utiliza_biblioteca_escola,rede,id_escola,serie,turno,desempenho_aluno_cat
274,2015,PB,1,17518688,0,MT,-0.472096,0.538400,223.577837,30.092429,...,1.0,1.0,2.0,2.0,0.0,2,25077147,9,1,0
348,2015,PB,1,17518163,0,MT,-0.809452,0.580177,204.722224,32.427443,...,1.0,0.0,1.0,2.0,1.0,2,61135320,9,1,0
408,2015,PB,1,17491526,0,LP,-0.828752,0.366511,204.326255,20.192323,...,NaN,1.0,1.0,2.0,1.0,2,25034820,9,1,1
1594,2015,PB,1,17489545,0,MT,-0.069178,0.583999,246.097862,32.641064,...,1.0,1.0,0.0,2.0,0.0,2,25030620,9,2,1
1847,2015,PB,1,17561474,0,MT,-0.462874,0.543093,224.093276,30.354732,...,2.0,1.0,NaN,2.0,2.0,2,25120760,9,3,0


In [66]:
print(len(df))

print(len(df.dropna()))

df = df.dropna()

84174
50356


In [67]:
# valores unicos para cada variável, menos as de exclude
exclude = ['ano', 'sigla_uf', 'id_aluno', 'situacao_censo', 'disciplina', 'proficiencia', 'erro_padrao', 'proficiencia_saeb',
       'erro_padrao_saeb', 'sexo', 'raca_cor','ano_nascimento', 'rede', 'id_escola', 'serie', 'turno', 'escola_publica', 'desempenho_aluno']

for col in df.columns:
    if col not in exclude:
        print(f'{col}: {df[col].unique()}')

idade: [18 17 16 15 14 13]
escolaridade_mae: [4. 1. 0. 2. 3. 5.]
mae_sabe_ler_escrever: [1 0]
mae_le: [1. 0.]
escolaridade_pai: [1. 3. 4. 2. 0. 5.]
pai_sabe_ler_escrever: [1. 0.]
pai_le: [1. 0.]
responsaveis_incentivam_realizacao_licao_casa: [1. 0.]
responsaveis_incentivam_estudos: [1. 0.]
responsaveis_incentivam_leitura: [1. 0.]
responsaveis_incentivam_comparecer_aulas: [1. 0.]
possui_computador: [1. 0. 2. 3. 4.]
tempo_lazer: [1. 4. 2. 3. 0.]
leitura_jornais: [1. 0. 2.]
leitura_livros_geral: [1. 2. 0.]
leitura_literatura_infantojuvenil: [1. 2. 0.]
leitura_historia_quadrinhos: [1. 2. 0.]
leitura_revista_comportamento: [1. 2. 0.]
leitura_revistas_geral: [1. 2. 0.]
leitura_internet: [0. 1. 2.]
frequenta_biblioteca: [1. 0. 2.]
frequenta_cinema: [0. 2. 1.]
frequenta_espetaculo_exposicao: [1. 2. 0.]
frequenta_festas_comunidade: [1. 2. 0.]
gosta_estudar_disciplina: [0. 1.]
faz_licao_casa: [1. 2. 0.]
correcao_licao_casa: [1. 2. 0.]
utiliza_biblioteca_escola: [1. 2. 0.]
desempenho_aluno_cat: [

# transforming to binary

In [68]:
# feature engineering para as colunas de leitura
# se o aluno ler em pelo menos 1 das categorias, 
# então a variável de leitura é 1, se não ler em todas, então 0

columns_oi = [col for col in df.columns if 'leitura_' in col]
print(df[columns_oi].sum(axis=1).apply(lambda x: 1 if x > 0 else 0).value_counts())
df['leitura'] = df[columns_oi].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

df = df.drop(columns=columns_oi)

1    49486
0      870
Name: count, dtype: int64


In [69]:
# colunas de 'frequenta'

columns_oi = [col for col in df.columns if 'frequenta_' in col]
print(df[columns_oi].sum(axis=1).apply(lambda x: 1 if x > 0 else 0).value_counts())
df['frequenta_entretenimento'] = df[columns_oi].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

df = df.drop(columns=columns_oi)

1    47175
0     3181
Name: count, dtype: int64


In [70]:
# tempo_lazer
# se o aluno tem tempo de lazer, então 1, se não, 0
df['tempo_lazer'] = df['tempo_lazer'].apply(lambda x: 1 if x > 0 else 0)

df['tempo_lazer'].value_counts()

tempo_lazer
1    48297
0     2059
Name: count, dtype: int64

In [71]:
# responsaveis incentivam
# mesma lógica da anterior, se o responsável incentiva em pelo menos 1 das categorias,
# então a variável de incentivo_responsaveis é 1, se não incentiva em todas, então 0

columns_oi = [col for col in df.columns if 'responsaveis_incentivam' in col]
print(df[columns_oi].sum(axis=1).apply(lambda x: 1 if x > 0 else 0).value_counts())

df['incentivo_responsaveis'] = df[columns_oi].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

df = df.drop(columns=columns_oi)

1    50208
0      148
Name: count, dtype: int64


In [72]:
# escolaridade pai / mae
# Nunca estudou.	Não completou a 4.ª série/5.º ano.	Completou a 4.ª série/5.º ano, mas não completou a 8.ª série/9.º ano.	Completou a 8.ª série/9.º ano, mas não completou o Ensino Médio.	Completou o Ensino Médio, mas não completou a Faculdade.	Completou a Faculdade.
# 0	1	2	3	4	5

# separando em 
# Baixa escolaridade: 0, 1, 2 (0)
# "Alta" escolaridade: 3, 4, 5 (1)

columns_oi = [col for col in df.columns if 'escolaridade_' in col]
print(df[columns_oi].sum(axis=1).apply(lambda x: 1 if x > 2 else 0).value_counts())

df['escolaridade_pai'] = df['escolaridade_pai'].apply(lambda x: 1 if x > 2 else 0)
df['escolaridade_mae'] = df['escolaridade_mae'].apply(lambda x: 1 if x > 2 else 0)

1    38588
0    11768
Name: count, dtype: int64


In [73]:
# possui_computador
# Na sua casa tem computador?	Não tem.	Sim, um.	Sim, dois.	Sim, três.	Sim, quatro ou mais.

# separando em possui, ou não

columns_oi = ['possui_computador']
print(df[columns_oi].sum(axis=1).apply(lambda x: 1 if x > 0 else 0).value_counts())

df['possui_computador'] = df[columns_oi].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

0    25411
1    24945
Name: count, dtype: int64


In [74]:
# faz_licao_casa
# correcao_licao_casa
# utiliza_biblioteca_escola

# mesma lógica das anteriores
columns_oi = ['faz_licao_casa', 'correcao_licao_casa', 'utiliza_biblioteca_escola']

df['faz_licao_casa'] = df['faz_licao_casa'].apply(lambda x: 1 if x > 0 else 0)
df['correcao_licao_casa'] = df['correcao_licao_casa'].apply(lambda x: 1 if x > 0 else 0)
df['utiliza_biblioteca_escola'] = df['utiliza_biblioteca_escola'].apply(lambda x: 1 if x > 0 else 0)

for column in columns_oi:
    print(df[column].value_counts())
    print('\n')

faz_licao_casa
1    47285
0     3071
Name: count, dtype: int64


correcao_licao_casa
1    48337
0     2019
Name: count, dtype: int64


utiliza_biblioteca_escola
1    30352
0    20004
Name: count, dtype: int64




In [75]:
for column in df.columns:
    if column not in exclude:
        print(f'{column}: {df[column].unique()}')

idade: [18 17 16 15 14 13]
escolaridade_mae: [1 0]
mae_sabe_ler_escrever: [1 0]
mae_le: [1. 0.]
escolaridade_pai: [0 1]
pai_sabe_ler_escrever: [1. 0.]
pai_le: [1. 0.]
possui_computador: [1 0]
tempo_lazer: [1 0]
gosta_estudar_disciplina: [0. 1.]
faz_licao_casa: [1 0]
correcao_licao_casa: [1 0]
utiliza_biblioteca_escola: [1 0]
desempenho_aluno_cat: [0 1 2 3]
leitura: [1 0]
frequenta_entretenimento: [1 0]
incentivo_responsaveis: [1 0]


In [76]:
save = False
if save:
    df.to_pickle('data/saeb_binary.pkl')